In [ ]:
# 1. Imports
import os
import cv2
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
def load_images_and_video_frames(folder, label):
    data = []
    for root, dirs, files in os.walk(folder):
        for filename in files:
            filepath = os.path.join(root, filename)
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = cv2.imread(filepath)
                if img is not None:
                    img = cv2.resize(img, (224, 224))
                    data.append((img, label))
            elif filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                cap = cv2.VideoCapture(filepath)
                frame_count = 0
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    if frame_count % 30 == 0:  # Sample every 30th frame
                        frame = cv2.resize(frame, (224, 224))
                        data.append((frame, label))
                    frame_count += 1
                cap.release()
    return data

# Set your training directories
vehicles_folder = r"C:\Users\HP\Desktop\Training\Vehicles"
cyclists_folder = r"C:\Users\HP\Desktop\Training\Cyclists"

# Load data for each class
vehicles_data = load_images_and_video_frames(vehicles_folder, 0)
cyclists_data = load_images_and_video_frames(cyclists_folder, 1)

# Print counts for each class
print(f"Loaded {len(vehicles_data)} images/frames for Vehicles.")
print(f"Loaded {len(cyclists_data)} images/frames for Cyclists.")

# Combine the data
data = vehicles_data + cyclists_data

print(f"Total loaded: {len(data)} images/frames for training.")

Loaded 1401 images/frames for Vehicles.
Loaded 13674 images/frames for Cyclists.
Total loaded: 15075 images/frames for training.


In [2]:
# 3. Data Preparation
X = np.array([img for img, label in data])
y = np.array([label for img, label in data])

NameError: name 'np' is not defined

In [8]:
# 4. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#5. Data Loader
X_train_tensor = torch.FloatTensor(X_train.transpose(0, 3, 1, 2)) / 255.0
X_test_tensor = torch.FloatTensor(X_test.transpose(0, 3, 1, 2)) / 255.0
y_train_tensor = torch.FloatTensor(y_train)
y_test_tensor = torch.FloatTensor(y_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
# 6. Model Definition
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 54 * 54, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 54 * 54)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = CNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
print(model)

In [ ]:
# 7. Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels.view(-1, 1)).item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted.view(-1) == labels).sum().item()
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Training Loss: {running_loss/len(train_loader):.4f}')
    print(f'Validation Loss: {val_loss/len(test_loader):.4f}')
    print(f'Validation Accuracy: {100 * correct/total:.2f}%\\n')

In [ ]:
# 8. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted.view(-1) == labels).sum().item()

print(f'Test accuracy: {100 * correct/total:.2f}%')

In [ ]:
# 9. Save and Load Model
torch.save(model.state_dict(), 'vehicle_cyclist_classifier.pth')
# To load later:
# model = CNN()
# model.load_state_dict(torch.load('vehicle_cyclist_classifier.pth'))
# model.eval()

In [26]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 model (or YOLOv5 if you prefer)
model = YOLO('yolov8n.pt')  # or 'yolov5s.pt'

# Run inference on an image
results = model('path/to/image.jpg')

# Count vehicles and cyclists
vehicle_classes = ['car', 'bus', 'truck']  # adjust as needed
cyclist_classes = ['bicycle', 'motorcycle']

vehicle_count = 0
cyclist_count = 0

for r in results:
    for c in r.boxes.cls:
        class_name = model.names[int(c)]
        if class_name in vehicle_classes:
            vehicle_count += 1
        elif class_name in cyclist_classes:
            cyclist_count += 1

print(f"Vehicles: {vehicle_count}, Cyclists: {cyclist_count}")

FileNotFoundError: path/to/image.jpg does not exist